# Create a Heater

In [6]:
import pyomo.environ as pe
from pyomo.common.config import ConfigBlock, ConfigValue, In
from idaes.core import (ControlVolume0DBlock,
                        declare_process_block_class,
                        EnergyBalanceType,
                        MomentumBalanceType,
                        MaterialBalanceType,
                        UnitModelBlockData,
                        useDefault,
                        FlowsheetBlock)
from idaes.core.util.config import is_physical_parameter_block
from idaes.examples.workshops.Module_3_Custom_Unit_Model.methanol_param_VLE import PhysicalParameterBlock
from idaes.core.util.misc import add_object_reference

In [7]:
def make_control_volume(unit, name, config):
    if config.dynamic is not False:
        raise ValueError('IdealGasIsentropcCompressor does not support dynamics')
    if config.has_holdup is not False:
        raise ValueError('IdealGasIsentropcCompressor does not support holdup')

    control_volume = ControlVolume0DBlock(default={"property_package": config.property_package,
                                                   "property_package_args": config.property_package_args})

    setattr(unit, name, control_volume)

    control_volume.add_state_blocks(has_phase_equilibrium=config.has_phase_equilibrium)
    control_volume.add_material_balances(balance_type=config.material_balance_type,
                                         has_phase_equilibrium=config.has_phase_equilibrium)
    control_volume.add_total_enthalpy_balances(has_heat_of_reaction=False, 
                                               has_heat_transfer=True, 
                                               has_work_transfer=False)
    control_volume.add_total_pressure_balances(has_pressure_change=False)

In [8]:
def make_config_block(config):
    config.declare("material_balance_type",
        ConfigValue(default=MaterialBalanceType.componentPhase, domain=In(MaterialBalanceType)))
    config.declare("energy_balance_type",
        ConfigValue(default=EnergyBalanceType.enthalpyTotal, domain=In([EnergyBalanceType.enthalpyTotal])))
    config.declare("momentum_balance_type",
        ConfigValue(default=MomentumBalanceType.pressureTotal, domain=In([MomentumBalanceType.pressureTotal])))
    config.declare("has_phase_equilibrium",
        ConfigValue(default=False, domain=In([False])))
    config.declare("has_pressure_change",
        ConfigValue(default=False, domain=In([False])))
    config.declare("property_package",
        ConfigValue(default=useDefault, domain=is_physical_parameter_block))
    config.declare("property_package_args",
        ConfigBlock(implicit=True))

In [9]:
@declare_process_block_class("Heater")
class HeaterData(UnitModelBlockData):
    CONFIG = UnitModelBlockData.CONFIG()
    make_config_block(CONFIG)

    def build(self):
        super(HeaterData, self).build()

        make_control_volume(self, "control_volume", self.config)

        self.add_inlet_port()
        self.add_outlet_port()
        
        add_object_reference(self, 'heat', self.control_volume.heat[0.0])

In [10]:
m = pe.ConcreteModel()
m.fs = fs = FlowsheetBlock(default={"dynamic": False})
fs.properties = props = PhysicalParameterBlock(default={'Cp': 0.038056, 'valid_phase': 'Vap'})

fs.heater = Heater(default={"property_package": props, 'has_phase_equilibrium': False})
fs.heater.inlet.flow_mol.fix(1)
fs.heater.inlet.mole_frac[0, 'CH3OH'].fix(0.25)
fs.heater.inlet.mole_frac[0, 'CH4'].fix(0.25)
fs.heater.inlet.mole_frac[0, 'H2'].fix(0.25)
fs.heater.inlet.mole_frac[0, 'CO'].fix(0.25)
fs.heater.inlet.pressure.fix(0.1)
fs.heater.inlet.temperature.fix(3)
fs.heater.heat.fix(5)

opt = pe.SolverFactory('ipopt')
res = opt.solve(m, tee=True)
print(res.solver.termination_condition)
fs.heater.outlet.display()

2019-06-29 14:32:40 - ERROR - idaes.core.process_block - Failure in build: fs.heater.control_volume.properties_in[0.0]
Traceback (most recent call last):
  File "/home/dang/src/idaes/dangunter/idaes-dev/idaes/core/process_block.py", line 38, in _rule_default
    b.build()
  File "/home/dang/miniconda3/lib/python3.6/site-packages/pyomo/core/base/block.py", line 549, in __getattr__
    % (self.__class__.__name__, val))
AttributeError: '_BlockData' object has no attribute 'build'
ERROR: Constructing component 'fs.heater.control_volume.properties_in' from
    data=None failed: AttributeError: '_BlockData' object has no attribute
    'build'
2019-06-29 14:32:40 - ERROR - idaes.core.process_block - Failure in build: fs.heater
Traceback (most recent call last):
  File "/home/dang/src/idaes/dangunter/idaes-dev/idaes/core/control_volume0d.py", line 135, in add_state_blocks
    default=tmp_dict))
  File "/home/dang/miniconda3/lib/python3.6/site-packages/pyomo/core/base/block.py", line 568, in __

PropertyPackageError: fs.heater.control_volume physical property package has not implemented the state_block_class attribute. Please contact the developer of the physical property package.

In [11]:
# For testing purposes
from pyomo.environ import TerminationCondition
assert res.solver.termination_condition == TerminationCondition.optimal

NameError: name 'res' is not defined